In [ ]:
""" mlist2label
Generating Frames using mlist from traditional method
"""


import torch
import torch.nn.functional as F
from torch import Tensor
import os
import tifffile
import scipy.io
import tqdm


# dimensional config
dim_frame = Tensor([ 32, 512, 512]).int()   # (C, H, W), (130, 130, 130)nm
up_sample = Tensor([  4,   8,   8]).int()   # (C, H, W)
dim_label = dim_frame * up_sample
# whether using luminance information
lum_info  = True,               # by label *= frame
# data path
frames_load_fold = "D:/frames"
mlists_load_fold = "D:/mlists"
data_save_fold   = "D:/temp"


# file name list
frames_list = os.listdir(frames_load_fold)
mlists_list = os.listdir(mlists_load_fold)

# create folder
if not os.path.exists(data_save_fold): os.makedirs(data_save_fold)

result = torch.zeros(*dim_label.tolist())
for index in tqdm.tqdm(
    range(len(frames_list)), desc=data_save_fold, unit="frame"
):
    # frame
    frame = torch.from_numpy(tifffile.imread(
        os.path.join(frames_load_fold, frames_list[index])
    ))
    # we got 6.5 by averging the max value of all frames
    frame = (frame / 6.5).float()
    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        size = (32, 512, 512)
    ).squeeze(0).squeeze(0)
    frame = torch.clip(frame, 0, 1)

    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        scale_factor=up_sample.tolist()
    ).squeeze(0).squeeze(0)

    # mlist
    _, mlist = scipy.io.loadmat(
        os.path.join(mlists_load_fold, mlists_list[index])
    ).popitem()
    mlist = torch.from_numpy(mlist).float()
    mlist = mlist[:, [2, 0, 1]] - 1  # (H W D) -> (D H W)
    mlist[:, 0] = (mlist[:, 0] + 0.5) / 2 - 0.5

    mlist = (mlist + 0.5) * up_sample - 0.5
    mlist = torch.round(mlist).int()

    # label
    label = torch.zeros(*dim_label.tolist())
    label[tuple(mlist.t())] = 1
    if lum_info: label *= frame
    result += label

    # save after combine 1, 2, 4, 8, 16... frames
    current_frame = index + 1
    if current_frame & (current_frame - 1) == 0 or index == 45000 - 1:
        tifffile.imwrite(
            "{}/{:05}.tif".format(data_save_fold, current_frame),
            result.numpy()
        )

In [ ]:
""" resolution
Measure the resolution by cluster.
"""


import numpy as np
from sklearn.mixture import GaussianMixture
import scipy.io
import os
import tqdm
from typing import List


def get_cluster_mlist(
    clsuter_list: np.ndarray, wide: int, mlists_load_fold: str
) -> List[np.ndarray]:
    # file name list
    mlists_list = os.listdir(mlists_load_fold)

    # use to store final result
    cluster_mlist = [[] for _ in range(len(clsuter_list))]

    # go through all mlists and get the mlist for each cluster
    for index in tqdm.tqdm(range(45000), unit="mlist"):
        # read a mlist out of 45000 mlists
        _, mlist = scipy.io.loadmat(
            os.path.join(mlists_load_fold, mlists_list[index])
        ).popitem()
        # (H W D), index start from 1 -> (D H W), index start from 0
        mlist = mlist[:, [2, 0, 1]] - 1
        # [64 512 512] -> [32 512 512]
        mlist[:, 0] = (mlist[:, 0] + 0.5) / 2 - 0.5

        for c in range(len(clsuter_list)):
            # find the submlist that match the position with current cluster
            position = clsuter_list[c]  # (D H W)
            submlist = mlist
            submlist = submlist[submlist[:, 0] >= position[0] - wide]
            submlist = submlist[submlist[:, 0] <= position[0] + wide]
            submlist = submlist[submlist[:, 1] >= position[1] - wide]
            submlist = submlist[submlist[:, 1] <= position[1] + wide]
            submlist = submlist[submlist[:, 2] >= position[2] - wide]
            submlist = submlist[submlist[:, 2] <= position[2] + wide]

            # add the molecular in submlist to cluster_mlist
            for i in range(len(submlist)): cluster_mlist[c].append(submlist[i])

    # mena center each cluster's mlist
    for k in range(len(cluster_mlist)):
        cluster_mlist[k] = np.array(cluster_mlist[k])
        if cluster_mlist[k].shape[0] == 0: continue

        gmm = GaussianMixture(n_components=1)
        gmm.fit(cluster_mlist[k])
        cluster_mlist[k] -= gmm.means_[0]

    return cluster_mlist  # [K, num of mol in each cluster]


# load cluster position
cluster_list = np.array([  # (D H W), index start from 0
    [85, 1603, 523 ],
    [88, 1437, 512 ],
    [86, 1447, 493 ],
    [51, 1546, 685 ],
    [53, 1576, 867 ],
    [57, 1768, 1110],
    [58, 1686, 1214],
    [61, 670 , 482 ],
    [59, 574 , 607 ],
    [62, 531 , 1239],
    [62, 712 , 1416],
    [61, 1059, 1671]
])
# since position in cluster_list find in super resolution image
cluster_list = (cluster_list + 0.5) / 4 - 0.5

# go through all mlists and get the mlist (mean centered) for each cluster
cluster_mlist = get_cluster_mlist(cluster_list, 0.5, "D:/mlists")

# combine all cluster's mlist and fit a gmm
all_mlist = []
for k in range(len(cluster_mlist)): all_mlist.extend(cluster_mlist[k])
gmm = GaussianMixture(n_components=1)
gmm.fit(all_mlist)

# information
print(gmm.covariances_)
print(gmm.means_)

In [2]:
""" 64 512 512 -> 32 512 512
Convert the Xunamen output 
from (64 512 512)pixel ( 65 130 130)nm 16bit
to   (32 512 512)pixel (130 130 130)nm 16bit.
"""


import torch
import torch.nn.functional as F
import os
import tifffile
import tqdm


frames_load_fold = "E:/frames"
frames_save_fold = "D:/frames"


frames_list = os.listdir(frames_load_fold)
for index in tqdm.tqdm(range(len(frames_list))):
    # read 16 bit tiff, note that the order of os.listdir is not guaranteed
    frame = torch.from_numpy(tifffile.imread(
        os.path.join(frames_load_fold, frames_list[index])
    )).float()
    # 64 512 512 -> 32 512 512
    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        size = (32, 512, 512)
    ).squeeze(0).squeeze(0).half()
    # save 16 bit with file name formation
    tifffile.imwrite(
        "{}/{:05}.tif".format(frames_save_fold, int(frames_list[index][11:-4])),
        frame.numpy()
    )

100%|██████████| 45000/45000 [58:45<00:00, 12.76it/s]  
